In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
import altair as alt
alt.data_transformers.enable("json")
import act
import glob
import xarray as xr
from sublimpy import utils

# Open data

Radiosonde synoptic winds

In [ ]:
df1 = pd.read_parquet("../sail/synoptic_winds_500_local.parquet").assign(pressure = 500)
df2 = pd.read_parquet("../sail/synoptic_winds_700_local.parquet").assign(pressure = 700)
df1.time = df1.time.apply(lambda dt: dt.replace(minute=0, second=0))
df2.time = df2.time.apply(lambda dt: dt.replace(minute=0, second=0))
synopticwinds_df = pd.concat([df1, df2])
synopticwinds_df = synopticwinds_df.groupby(['time', 'pressure']).mean().reset_index()

In [ ]:
synopticwinds_df

In [ ]:
alt.Chart(
    synopticwinds_df
).mark_bar().encode(
    alt.X("wspd:Q").bin(True, maxbins=30),
    alt.Y("count():Q")
)

Doppler Lidar (aggregated) data

In [ ]:
dl_df = pd.read_parquet("/Users/elischwat/Development/data/sublimationofsnow/sail_processed/gucdlrhiM1.b1/").set_index([
    'z_binned',	'x_offset',	'scan_time'
])['streamwise_velocity'].reset_index()

dl_df = dl_df[dl_df.scan_time >= "2023-02-09 0000"]


# Plot vertical profiles over a day

In [274]:
src = dl_df[dl_df.scan_time.dt.date == dt.date(2023,4,16)].query("x_offset == -250")
src = src[src.z_binned < 800]
alt.Chart(src).mark_line().encode(
    alt.X("median:Q").title("Downvalley wind (m/s)"),
    alt.Y("z_binned:Q").title("Height (m, agl)"),
    alt.Color("minutes(scan_time):O").scale(scheme='turbo'),
    alt.Facet("hours(scan_time):T", columns=6),
    order='z_binned'
).properties(width = 125, height = 125, title = f"Along-valley wind profiles on {str(src.scan_time.dt.date.iloc[0])}")

alt.Chart(...)

# Radiosonde data

In [297]:
SONDE_DATA_STREAM = 'gucsondewnpnM1.b1'
SONDE_DATA_STREAM_FILEEXT = '.cdf'
SONDE_OUTPUT_DIR = os.path.join("/Users/elischwat/Development/data/sublimationofsnow/", SONDE_DATA_STREAM)

In [298]:
start_date = "20230201"
end_date = "20230619"

In [ ]:
act.discovery.download_arm_data(
            username, token, SONDE_DATA_STREAM, start_date, end_date, output = SONDE_OUTPUT_DIR)

In [299]:
sonde_files = glob.glob(os.path.join(SONDE_OUTPUT_DIR,'*'+SONDE_DATA_STREAM_FILEEXT))
print(len(sonde_files))

395


In [300]:
sonde = act.io.arm.read_arm_netcdf(sonde_files)
sonde = sonde.where(sonde.pres > 450).where(sonde.pres < 550)
sonde = utils.modify_xarray_timezone(sonde, 'UTC', 'US/Mountain')
sonde_daily_mean = sonde.resample(time='1D').mean()

In [668]:
sonde_daily_mean.sel(time = slice('20230415')).to_dataframe().wspd.mean()

20.943981

In [667]:
sonde_daily_mean.sel(time = slice('20230418')).to_dataframe().wspd.mean()

20.883871

In [301]:
sonde_daily_mean_df = sonde_daily_mean.to_dataframe()

In [302]:
bar_chart = alt.Chart(sonde_daily_mean_df).mark_bar().encode(
    alt.X('wspd:Q').bin().title('500 hPa wind speed, daily mean (m/s)'),
    alt.Y('count()').title('Number of days')
).properties(width = 250, height = 125)
median_line = alt.Chart(sonde_daily_mean_df).mark_rule(color='orange', strokeWidth=3).encode(
    alt.X('median(wspd):Q'),
).properties(title = f"{str(round(sonde_daily_mean_df.wspd.median(), 1))[:4]}")
bar_chart + median_line

alt.LayerChart(...)

In [330]:
days_with_low_synoptic_wind_speeds = sonde_daily_mean_df[
    sonde_daily_mean_df.wspd <= sonde_daily_mean_df.wspd.median()
].index.date

In [580]:
sonde_daily_mean[]

<xarray.Dataset> Size: 26kB
Dimensions:    (time: 198)
Coordinates:
  * time       (time) datetime64[ns] 2kB 2022-11-30 2022-12-01 ... 2023-06-15
Data variables: (12/22)
    pres       (time) float32 792B dask.array<chunksize=(1,), meta=np.ndarray>
    qc_pres    (time) float64 2kB dask.array<chunksize=(1,), meta=np.ndarray>
    tdry       (time) float32 792B dask.array<chunksize=(1,), meta=np.ndarray>
    qc_tdry    (time) float64 2kB dask.array<chunksize=(1,), meta=np.ndarray>
    dp         (time) float32 792B dask.array<chunksize=(1,), meta=np.ndarray>
    qc_dp      (time) float64 2kB dask.array<chunksize=(1,), meta=np.ndarray>
    ...         ...
    wstat      (time) float32 792B dask.array<chunksize=(1,), meta=np.ndarray>
    asc        (time) float32 792B dask.array<chunksize=(1,), meta=np.ndarray>
    qc_asc     (time) float64 2kB dask.array<chunksize=(1,), meta=np.ndarray>
    lat        (time) float32 792B dask.array<chunksize=(1,), meta=np.ndarray>
    lon        (time) float32 792B dask.array<chunksize=(1,), meta=np.ndarray>
    alt        (time) float32 792B dask.array<chunksize=(1,), meta=np.ndarray>
Attributes: (12/36)
    command_line:               sonde_ingest -s guc -f M1
    Conventions:                ARM-1.2
    process_version:            ingest-sonde-10.14-1.el7
    dod_version:                sondewnpn-b1-2.3
    input_source:               /data/collection/guc/gucsondeM1.00/good.2022N...
    site_id:                    guc
    ...                         ...
    doi:                        10.5439/1595321
    history:                    created by user dsmgr on machine prod-proc2.a...
    _file_dates:                ['20221130', '20221130', '20221201', '2022120...
    _file_times:                ['112900', '232900', '113100', '232600', '113...
    _datastream:                gucsondewnpnM1.b1
    _arm_standards_flag:        1

# Plot monthly-averaged vertical profiles

## All data

In [307]:
dl_df.query("z_binned == 15.0").query("x_offset == -250").groupby(dl_df.scan_time.dt.month).count()

,z_binned,x_offset,scan_time,mean,median,std,count
scan_time,,,,,,,
2,1758,1758,1758,1735,1735,1641,1758
3,2972,2972,2972,2906,2906,2725,2972
4,2880,2880,2880,2871,2871,2836,2880
5,2976,2976,2976,2976,2976,2961,2976
6,1485,1485,1485,1485,1485,1485,1485


In [308]:
src = dl_df.query("x_offset == -250").dropna()
src = src.dropna()
src['month'] = src['scan_time'].dt.month
src['hour'] = src['scan_time'].dt.hour

src = src.groupby(['month', 'hour', 'z_binned'])[
    ['mean', 'median', 'std']
].agg(
    {'mean': 'mean', 'median': 'median', 'std': 'mean'}
).reset_index()


In [340]:
src = dl_df.query("x_offset == -250").dropna()
src = src.dropna()
src['month'] = src['scan_time'].dt.month
src['hour'] = src['scan_time'].dt.hour
src['hour_group'] = pd.cut(
    src['hour'], 
    [-1,2.5, 5.5, 8.5, 11.5, 14.5, 17.5, 20.5, 23.5],
    labels=['00-02', '03-05', '06-08', '09-11', '12-14', '15-17', '18-20', '21-23']
)

src = src.groupby(['month', 'hour_group', 'z_binned'])[
    ['mean', 'median', 'std']
].agg(
    {'mean': 'mean', 'median': 'median', 'std': 'mean'}
).reset_index()

alt.Chart(
    src.query("z_binned <= 625")
).mark_line().encode(
    alt.X("mean:Q").title("Downvalley wind (m/s)"),
    alt.Y("z_binned:Q").title("Height (m, agl)"),
    alt.Color("month:O").scale(scheme='turbo'),
    alt.Facet("hour_group", columns=4).title("hours of day"),
    order='z_binned'
).properties(
    width = 125, 
    height = 125,
    title='Mean vertical profiles of valley wind for different parts of the day'
).display(renderer='svg')

/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_49859/1909133328.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  src = src.groupby(['month', 'hour_group', 'z_binned'])[


alt.Chart(...)

## Weak synoptic days only

In [424]:
print(2, ':', len([d for d in days_with_low_synoptic_wind_speeds if d.month == 2]))
print(3, ':', len([d for d in days_with_low_synoptic_wind_speeds if d.month == 3]))
print(4, ':', len([d for d in days_with_low_synoptic_wind_speeds if d.month == 4]))
print(5, ':', len([d for d in days_with_low_synoptic_wind_speeds if d.month == 5]))
print(6, ':', len([d for d in days_with_low_synoptic_wind_speeds if d.month == 6]))

2 : 13
3 : 11
4 : 16
5 : 27
6 : 15


In [576]:
src = dl_df.query("x_offset == -250").dropna()
src = src.dropna()
src['month'] = src['scan_time'].dt.month
src['hour'] = src['scan_time'].dt.hour
src['hour_group'] = pd.cut(
    src['hour'], 
    [-1,2.5, 5.5, 8.5, 11.5, 14.5, 17.5, 20.5, 23.5],
    labels=['00-02', '03-05', '06-08', '09-11', '12-14', '15-17', '18-20', '21-23']
)

src = src[src.scan_time.dt.date.isin(days_with_low_synoptic_wind_speeds)]

src = src.groupby(['month', 'hour_group', 'z_binned'])[
    ['mean', 'median', 'std']
].agg(
    {'mean': 'mean', 'median': 'median', 'std': 'mean'}
).reset_index()

alt.Chart(
    src.query("z_binned <= 625")
).mark_line().encode(
    alt.X("median:Q").title("Downvalley wind (m/s)"),
    alt.Y("z_binned:Q").title("Height (m, agl)"),
    alt.Color("month:O").scale(scheme='turbo'),
    alt.Facet("hour_group", columns=4).title("hours of day"),
    order='z_binned'
).properties(
    width = 125, 
    height = 125,
    title='Mean vertical profiles of valley wind for different parts of the day'
).display(renderer='svg')

/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_49859/4124793581.py:13: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  src = src.groupby(['month', 'hour_group', 'z_binned'])[


alt.Chart(...)

In [579]:

alt.Chart(
    src[src.hour_group.isin(['00-02', '09-11', '15-17'])].query("z_binned <= 625")
).mark_line().encode(
    alt.X("median:Q").title("Downvalley wind (m/s)"),
    alt.Y("z_binned:Q").title("Height (m, agl)"),
    alt.Color("month:O").scale(scheme='turbo'),
    alt.Facet("hour_group", columns=4).title("hours of day"),
    order='z_binned'
).properties(
    width = 125, 
    height = 125,
    title='Mean vertical profiles of valley wind for different parts of the day'
).display(renderer='svg')

alt.Chart(...)

# Case study plots

In [556]:
src = dl_df[dl_df.x_offset == -250]
src = src[src.scan_time.dt.date.isin([
    dt.date(2023, 4, 16),
    dt.date(2023, 6, 9)
])]
src = src.query("z_binned <= 650")
src['z_binned'] = src['z_binned'].replace({625: 600})
src = src.set_index('scan_time').sort_index()
src = src.groupby([pd.Grouper(freq='120Min'), 'z_binned'])[['median', 'std']].agg({'median': 'median', 'std': 'mean'}).reset_index()
src['date_str'] = src['scan_time'].dt.date.astype(str)
src['hour'] = src['scan_time'].dt.hour
src['time_category'] = pd.cut(
    src['scan_time'].dt.hour,
    [-0.5, 6.5, 14.5, 23.5],
    labels = ['Morning (0000-0600)', 'Day (0800-1400)', 'Evening (1600-2200)']
)
src['lower_bound'] = src['median'] - 2*src['std']
src['upper_bound'] = src['median'] + 2*src['std']

error_bands_chart = alt.Chart(src).mark_errorband().encode(
    alt.X("lower_bound:Q").title("Downvalley velocity (m/s)"),
    alt.X2("upper_bound:Q").title("Downvalley velocity (m/s)"),
    alt.Y("z_binned:Q").title("Height (m, agl)"),
    alt.Color("hour:O").scale(scheme='rainbow'),
).properties(width = 150, height = 150)

lines_chart = alt.Chart(src).mark_line().encode(
    alt.X("median:Q").title("Downvalley velocity (m/s)"),
    alt.Y("z_binned:Q").title("Height (m, agl)"),
    alt.Color("hour:O").scale(scheme='rainbow'),
    order='z_binned'
).properties(width = 150, height = 150)


(error_bands_chart + lines_chart).facet(
    row = alt.Row("date_str:N").title(None).header(labelFontSize=16, labelFontWeight='bold'),
    column = alt.Column("time_category:O").sort(
        ['Morning (0000-0600)', 'Day (0800-1400)', 'Evening (1600-2200)']
    ).title(None).header(labelFontSize=16, labelFontWeight='bold'),
)

alt.FacetChart(...)

In [555]:
src = dl_df[dl_df.x_offset == -250]
src = src[src.scan_time.dt.date.isin([
    dt.date(2023, 3, 26),
    dt.date(2023, 6, 10)
])]
src = src.query("z_binned <= 650")
src['z_binned'] = src['z_binned'].replace({625: 600})
src = src.set_index('scan_time').sort_index()
src = src.groupby([pd.Grouper(freq='120Min'), 'z_binned'])[['median', 'std']].agg({'median': 'median', 'std': 'mean'}).reset_index()
src['date_str'] = src['scan_time'].dt.date.astype(str)
src['hour'] = src['scan_time'].dt.hour
src['time_category'] = pd.cut(
    src['scan_time'].dt.hour,
    [-0.5, 6.5, 14.5, 23.5],
    labels = ['Morning (0000-0600)', 'Day (0800-1400)', 'Evening (1600-2200)']
)
src['lower_bound'] = src['median'] - 2*src['std']
src['upper_bound'] = src['median'] + 2*src['std']

error_bands_chart = alt.Chart(src).mark_errorband().encode(
    alt.X("lower_bound:Q").title("Downvalley velocity (m/s)").scale(domain=[-10,10], clamp=True),
    alt.X2("upper_bound:Q").title("Downvalley velocity (m/s)"),
    alt.Y("z_binned:Q").title("Height (m, agl)"),
    alt.Color("hour:O").scale(scheme='rainbow'),
).properties(width = 150, height = 150)

lines_chart = alt.Chart(src).mark_line().encode(
    alt.X("median:Q").title("Downvalley velocity (m/s)").scale(domain=[-10,10], clamp=True),
    alt.Y("z_binned:Q").title("Height (m, agl)"),
    alt.Color("hour:O").scale(scheme='rainbow'),
    order='z_binned'
).properties(width = 150, height = 150)


(
    error_bands_chart + 
    lines_chart).facet(
    row = alt.Row("date_str:N").title(None).header(labelFontSize=16, labelFontWeight='bold'),
    column = alt.Column("time_category:O").sort(
        ['Morning (0000-0600)', 'Day (0800-1400)', 'Evening (1600-2200)']
    ).title(None).header(labelFontSize=16, labelFontWeight='bold'),
)

alt.FacetChart(...)

In [605]:

date = dt.date(2023, 3, 26)
src = dl_df[dl_df.x_offset == -250]
src = src[src.scan_time.dt.date == date]
src = src.query("z_binned <= 650")
src['z_binned'] = src['z_binned'].replace({625: 600})
src = src.set_index('scan_time').sort_index()
src = src.groupby([pd.Grouper(freq='120Min'), 'z_binned'])[['median', 'std']].agg({'median': 'median', 'std': 'mean'}).reset_index()
src['date_str'] = src['scan_time'].dt.date.astype(str)
src['hour'] = src['scan_time'].dt.hour
src['time_category'] = pd.cut(
    src['scan_time'].dt.hour,
    [-0.5, 6.5, 14.5, 23.5],
    labels = ['Morning (0000-0600)', 'Day (0800-1400)', 'Evening (1600-2200)']
)
src['lower_bound'] = src['median'] - 2*src['std']
src['upper_bound'] = src['median'] + 2*src['std']
src = src[src.hour > 0]

lines_chart = alt.Chart(src).mark_line().encode(
    alt.X("median:Q").title("Downvalley velocity (m/s)").scale(domain=[-10,10], clamp=True),
    alt.Y("z_binned:Q").title("Height (m, agl)"),
    alt.Color("hour:O").scale(scheme='rainbow'),
    order='z_binned',
    tooltip = 'hour'
).properties(width = 150, height = 150)


facet_lines_chart = lines_chart.facet(
    row = alt.Row("date_str:N").title(None).header(labelFontSize=16, labelFontWeight='bold'),
    column = alt.Column("time_category:O").sort(
        ['Morning (0000-0600)', 'Day (0800-1400)', 'Evening (1600-2200)']
    ).title(None).header(labelFontSize=16, labelFontWeight='bold'),
)
display(facet_lines_chart)

alt.FacetChart(...)

In [581]:

date = dt.date(2023, 6, 10)
src = dl_df[dl_df.x_offset == -250]
src = src[src.scan_time.dt.date == date]
src = src.query("z_binned <= 650")
src['z_binned'] = src['z_binned'].replace({625: 600})
src = src.set_index('scan_time').sort_index()
src = src.groupby([pd.Grouper(freq='120Min'), 'z_binned'])[['median', 'std']].agg({'median': 'median', 'std': 'mean'}).reset_index()
src['date_str'] = src['scan_time'].dt.date.astype(str)
src['hour'] = src['scan_time'].dt.hour
src['time_category'] = pd.cut(
    src['scan_time'].dt.hour,
    [-0.5, 6.5, 14.5, 23.5],
    labels = ['Morning (0000-0600)', 'Day (0800-1400)', 'Evening (1600-2200)']
)
src['lower_bound'] = src['median'] - 2*src['std']
src['upper_bound'] = src['median'] + 2*src['std']
src = src[src.hour > 0]
src = src[src.hour.isin([2, 6, 10, 14])]

lines_chart = alt.Chart(src).mark_line().encode(
    alt.X("median:Q").title("Downvalley velocity (m/s)").scale(domain=[-10,10], clamp=True),
    alt.Y("z_binned:Q").title("Height (m, agl)"),
    alt.Color("hour:O").scale(scheme='rainbow'),
    order='z_binned',
    tooltip = 'hour'
).properties(width = 150, height = 150)


facet_lines_chart = lines_chart.facet(
    row = alt.Row("date_str:N").title(None).header(labelFontSize=16, labelFontWeight='bold'),
    column = alt.Column("time_category:O").sort(
        ['Morning (0000-0600)', 'Day (0800-1400)', 'Evening (1600-2200)']
    ).title(None).header(labelFontSize=16, labelFontWeight='bold'),
)
display(facet_lines_chart)

alt.FacetChart(...)

In [592]:

date = dt.date(2023, 6, 10)
src = dl_df[dl_df.x_offset == -250]
src = src[src.scan_time.dt.date == date]
src = src.query("z_binned <= 650")
src['z_binned'] = src['z_binned'].replace({625: 600})
src = src.set_index('scan_time').sort_index()
src = src.groupby([pd.Grouper(freq='120Min'), 'z_binned'])[['median', 'std']].agg({'median': 'median', 'std': 'mean'}).reset_index()
src['date_str'] = src['scan_time'].dt.date.astype(str)
src['hour'] = src['scan_time'].dt.hour
src['time_category'] = pd.cut(
    src['scan_time'].dt.hour,
    [-0.5, 6.5, 14.5, 23.5],
    labels = ['Morning (0000-0600)', 'Day (0800-1400)', 'Evening (1600-2200)']
)
src['lower_bound'] = src['median'] - 2*src['std']
src['upper_bound'] = src['median'] + 2*src['std']
src = src[src.hour > 0]
src = src[src.hour.isin([4, 14])]

lines_chart = alt.Chart(src).mark_line().encode(
    alt.X("median:Q").title("Downvalley velocity (m/s)").scale(domain=[-10,10], clamp=True),
    alt.Y("z_binned:Q").title("Height (m, agl)"),
    alt.Color("hour:O").scale(scheme='rainbow'),
    order='z_binned',
    tooltip = 'hour'
).properties(width = 150, height = 150)


lines_chart

alt.Chart(...)

# PPI scans

In [ ]:
ppi_output_dir = "/Users/elischwat/Development/data/sublimationofsnow/gucdlprofwind4newsM1.c1/"

In [ ]:
# username = os.getenv("ARM_USERNAME")
# token = os.getenv("ARM_TOKEN")

# act.discovery.download_arm_data(
#     username,    token,    'gucdlprofwind4newsM1.c1',    
#     '20230601', '20230619',
#     output = ppi_output_dir
# )

In [ ]:
ppi_ds = act.io.read_arm_netcdf(glob.glob(os.path.join(ppi_output_dir, "*.nc")))

In [ ]:
ppi_df = ppi_ds[['wind_direction', 'wind_speed']].to_dataframe()
ppi_df = ppi_df[ppi_df.index.get_level_values(1) <= 1000]
ppi_df = ppi_df.reset_index('height')
ppi_df_avg = ppi_df.groupby([pd.Grouper(freq='3H'), 'height']).median()


alt.Chart(ppi_df_avg.reset_index()).mark_line().encode(
    alt.X('median(wind_direction):Q').sort('-y'),
    alt.Y('height:Q'),
    alt.Color('hours(time):N').scale(scheme='rainbow'),
    alt.Facet('month(time):O',columns = 3),
)

# TROPoe scans

In [ ]:
tropoe_output_dir = "/Users/elischwat/Development/data/sublimationofsnow/guctropoeM1.c1/"

In [ ]:
username = os.getenv("ARM_USERNAME")
token = os.getenv("ARM_TOKEN")
act.discovery.download_arm_data(
    username,    token,    'guctropoeM1.c1',    
    '20230416', '20230417',
    output = tropoe_output_dir
)

In [387]:
# ds = xr.open_dataset("/Users/elischwat/Development/data/sublimationofsnow/guctropoeM1.c1/guctropoeM1.c1.20230505.000502.nc")
# ds = xr.open_dataset("/Users/elischwat/Development/data/sublimationofsnow/guctropoeM1.c1/guctropoeM1.c1.20230609.001002.nc")


In [418]:
ds = xr.open_dataset("/Users/elischwat/Development/data/sublimationofsnow/guctropoeM1.c1/guctropoeM1.c1.20230406.000502.nc")
df = ds[['theta']].to_dataframe().reset_index()
df = utils.modify_df_timezone(df, 'UTC', 'US/Mountain')
df['height'] = df['height']*1000
src = df.query("height < 2500").set_index('time').groupby(
        [pd.Grouper(freq='1H'), 'height']
    ).mean().reset_index()
src['height'] = src['height'].replace(0, 1)
src = src[src.time.dt.hour.isin([4, 8, 12, 16])]
alt.Chart(
    src
).mark_line(point=True, size=0.5).encode(
    alt.X('median(theta):Q').scale(zero=False),
    alt.Y('height').scale(type='linear'),
    alt.Color('hours(time):O').scale(scheme='rainbow'),
    order='height',
    tooltip = 'hours(time)'
).properties(width=250, height=250)

/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_49859/3908967017.py:6: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  [pd.Grouper(freq='1H'), 'height']


alt.Chart(...)

In [613]:
ds = xr.open_dataset("/Users/elischwat/Development/data/sublimationofsnow/guctropoeM1.c1/guctropoeM1.c1.20230326.001002.nc")

df = ds[['theta']].to_dataframe().reset_index()
df = utils.modify_df_timezone(df, 'UTC', 'US/Mountain')
df['height'] = df['height']*1000
src = df.query("height < 2500").set_index('time').groupby(
        [pd.Grouper(freq='1H'), 'height']
    ).mean().reset_index()
src['height'] = src['height'].replace(0, 1)
src = src[src.time.dt.hour.isin([4, 8, 11, 16])]
alt.Chart(
    src
).mark_line(point=True, size=0.5).encode(
    alt.X('median(theta):Q').scale(zero=False),
    alt.Y('height').scale(type='linear'),
    alt.Color('hours(time):O').scale(scheme='rainbow'),
    order='height',
    tooltip = 'hours(time)'
).properties(width=250, height=250, title='20230326')

/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_49859/367194871.py:7: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  [pd.Grouper(freq='1H'), 'height']


alt.Chart(...)

In [619]:
ds = xr.open_dataset("/Users/elischwat/Development/data/sublimationofsnow/guctropoeM1.c1/guctropoeM1.c1.20230610.000502.nc")
df = ds[['theta']].to_dataframe().reset_index()
df = utils.modify_df_timezone(df, 'UTC', 'US/Mountain')
df['height'] = df['height']*1000
src = df.query("height < 2500").set_index('time').groupby(
        [pd.Grouper(freq='1H'), 'height']
    ).mean().reset_index()
src['height'] = src['height'].replace(0, 1)
src = src[src.time.dt.hour.isin([4, 8, 12, 16])]
alt.Chart(
    src
).mark_line(point=True, size=0.5).encode(
    alt.X('median(theta):Q').scale(zero=False),
    alt.Y('height').scale(type='linear'),
    alt.Color('hours(time):O').scale(scheme='rainbow'),
    order='height',
    tooltip = 'hours(time)'
).properties(width=250, height=250, title='20230610')

/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_49859/3435149234.py:6: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  [pd.Grouper(freq='1H'), 'height']


alt.Chart(...)

In [646]:
ds = xr.open_dataset("/Users/elischwat/Development/data/sublimationofsnow/guctropoeM1.c1/guctropoeM1.c1.20230610.000502.nc")
df = ds[['theta']].to_dataframe().reset_index()
df = utils.modify_df_timezone(df, 'UTC', 'US/Mountain')
df['height'] = df['height']*1000
df = df[df.time < '20230610 1700'][df.time > '20230610  1500']
df = df.query('height < 2500')
src = df.set_index('time').groupby(
        [pd.Grouper(freq='15min'), 'height']
    ).mean().reset_index()
alt.Chart(
    src
).mark_line().encode(
    alt.X('theta:Q').scale(zero=False),
    alt.Y('height').scale(type='linear'),
    alt.Color('hoursminutes(time):O').scale(scheme='rainbow'),
    order='height',
    # tooltip = 'hours(time)'
).properties(width=250, height=250, title='20230610')

/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_49859/1812759104.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[df.time < '20230610 1700'][df.time > '20230610  1500']


alt.Chart(...)

In [618]:
ds = xr.open_dataset("/Users/elischwat/Development/data/sublimationofsnow/guctropoeM1.c1/guctropoeM1.c1.20230609.001002.nc")
df = ds[['theta']].to_dataframe().reset_index()
df = utils.modify_df_timezone(df, 'UTC', 'US/Mountain')
df['height'] = df['height']*1000
# src = df.query("height < 2500").set_index('time').groupby(
#         [pd.Grouper(freq='1H'), 'height']
#     ).mean().reset_index()
src['height'] = src['height'].replace(0, 1)
# src = src[src.time.dt.hour.isin([4, 8, 12, 16])]
alt.Chart(
    src
).mark_line(point=True, size=0.5).encode(
    alt.X('theta:Q').scale(zero=False),
    alt.Y('height').scale(type='linear'),
    alt.Color('hoursminutes(time):O').scale(scheme='rainbow'),
    order='height',
    tooltip = 'hours(time)'
).properties(width=250, height=250, title='20230609')

alt.Chart(...)

In [525]:
ds = xr.open_dataset("/Users/elischwat/Development/data/sublimationofsnow/guctropoeM1.c1/guctropoeM1.c1.20230608.001002.nc")
df = ds[['theta']].to_dataframe().reset_index()
df = utils.modify_df_timezone(df, 'UTC', 'US/Mountain')
df['height'] = df['height']*1000
src = df.query("height < 2500").set_index('time').groupby(
        [pd.Grouper(freq='1H'), 'height']
    ).mean().reset_index()
src['height'] = src['height'].replace(0, 1)
src = src[src.time.dt.hour.isin([4, 8, 14, 17])]
alt.Chart(
    src
).mark_line(point=True, size=0.5).encode(
    alt.X('median(theta):Q').scale(zero=False),
    alt.Y('height').scale(type='linear'),
    alt.Color('hours(time):O').scale(scheme='rainbow'),
    order='height',
    tooltip = 'hours(time)'
).properties(width=250, height=250, title='20230608')

/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_49859/3589463115.py:6: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  [pd.Grouper(freq='1H'), 'height']


alt.Chart(...)

# SOS Surface Winds

In [343]:
from sublimpy import utils

In [344]:
sos_tidy_fn = f"../paper1/process_slow_data/tidy_df_20221101_20230619_planar_fit_multiplane_STRAIGHTUP_q7_flags9000.parquet"

tidy_df = pd.read_parquet(sos_tidy_fn)

# Convert data timezone to local and clean up data on the ends
# convert time column to datetime
tidy_df['time'] = pd.to_datetime(tidy_df['time'])
tidy_df = utils.modify_df_timezone(tidy_df, 'UTC', 'US/Mountain')

In [350]:
src = tidy_df.query("variable == 'dir_5m_c'").query("time > '20230201'")

src = src[src.time.dt.date.isin(days_with_low_synoptic_wind_speeds)]

alt.Chart(
    src
).mark_line(point=True).encode(
    alt.X('hours(time):T'),
    alt.Y('median(value):Q'),
    alt.Color('month(time):N')
).properties(
    width=250, height=125
)

alt.Chart(...)

In [540]:
src = tidy_df.query("variable == 'dir_5m_c'").query("time > '20230201'")
src = src[src.time.dt.date == dt.date(2023, 3, 26)]
alt.Chart(
    src
).mark_line(point=True).encode(
    alt.X('hours(time):T'),
    alt.Y('median(value):Q'),
    alt.Color('month(time):N')
).properties(
    width=250, height=125
)

alt.Chart(...)

In [543]:
src = tidy_df.query("variable == 'dir_5m_c'").query("time > '20230201'")
src = src[src.time.dt.date == dt.date(2023,6,10)]
alt.Chart(
    src
).mark_line(point=True).encode(
    alt.X('hours(time):T'),
    alt.Y('median(value):Q'),
    alt.Color('month(time):N')
).properties(
    width=250, height=125
)

alt.Chart(...)

In [597]:
for d in range(1,32):
    src = tidy_df.query("variable == 'dir_5m_c'").query("time > '20230201'")
    date = dt.date(2023,3,d)
    src = src[src.time.dt.date == date]
    display(alt.Chart(
        src
    ).mark_line(point=True).encode(
        alt.X('hours(time):T'),
        alt.Y('median(value):Q'),
        alt.Color('month(time):N')
    ).properties(
        width=250, height=125, title=date.strftime('%m-%d')
    ))

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

...

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

In [ ]:
3, 23
9,18, 19, 28, 30

In [598]:
for d in range(1,31):
    src = tidy_df.query("variable == 'dir_5m_c'").query("time > '20230201'")
    date = dt.date(2023,4,d)
    src = src[src.time.dt.date == date]
    display(alt.Chart(
        src
    ).mark_line(point=True).encode(
        alt.X('hours(time):T'),
        alt.Y('median(value):Q'),
        alt.Color('month(time):N')
    ).properties(
        width=250, height=125, title=date.strftime('%m-%d')
    ))

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

...

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)